In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
from itertools import groupby

In [3]:
ruta = "C:/Users/9 ----- SIG/Documents/Excel para python SNIES/Desverificar.xlsx"
df = pd.read_excel(ruta)
df.head(3)

,ID_TIPO_DOCUMENTO,NUM_DOCUMENTO,PRO_CONSECUTIVO,NUM_ACTA_GRADO,FECHA_GRADO,NUM_FOLIO,Año,Semestre
0,CC,1106484032,5764,NaN,201201S,NaN,2012,1
1,CC,94405355,1943,NaN,200701S,NaN,2007,1
2,CC,91344550,12320,NaN,200701S,NaN,2007,1


In [4]:
# Crear el navegador
driver = webdriver.Chrome()
# Abrir la página de inicio de sesión
driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
time.sleep(1)
# Ingresar usuario y contraseña
usuario = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']")
usuario.send_keys("1826-admin")
contrasena = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']")
contrasena.send_keys("1826Uan9123")
# Clic en el botón de ingresar
boton_ingresar = driver.find_element(By.ID, "loginForm:submitLogin")
boton_ingresar.click()
# Ir a la sección administrador - verificar graduados
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_33"))).click()
time.sleep(0.3)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_37"))).click()
time.sleep(0.3)

# Inicializar columna de estado si no existe
if "ESTADO" not in df.columns:
    df["ESTADO"] = "Pendiente"

# Ordenar el DataFrame por Año, Semestre y PRO_CONSECUTIVO
df = df.sort_values(['Año', 'Semestre', 'PRO_CONSECUTIVO']).reset_index(drop=True)
print(f"📋 DataFrame ordenado por Año, Semestre y PRO_CONSECUTIVO - Total filas: {len(df)}")

# Agrupar por Año, Semestre y PRO_CONSECUTIVO
grupos = df.groupby(['Año', 'Semestre', 'PRO_CONSECUTIVO'])
print(f"📊 Total de grupos a procesar: {len(grupos)}")

grupo_num = 0
for (anio, semestre, pro_consecutivo), grupo in grupos:
    grupo_num += 1
    anio_excel = str(int(anio)).strip()
    semestre_excel = str(int(semestre)).strip()
    pro_consecutivo_excel = str(int(pro_consecutivo)).strip()
    
    print(f"\n🔄 Grupo {grupo_num}/{len(grupos)}: Año {anio_excel}, Semestre {semestre_excel}, PRO_CONSECUTIVO {pro_consecutivo_excel}")
    print(f"📋 Cédulas a procesar: {len(grupo)} documentos")
    
    cedulas_procesadas_grupo = 0
    cedulas_exitosas_grupo = 0
    
    try:
        # 1. Buscar y hacer clic en Año/Semestre con reintentos
        fila_encontrada = False
        for intento_anio in range(3):
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]"))
                )
                
                # Refrescar la búsqueda de filas en cada intento
                filas = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-hover')]//tr")
                
                for fila_html in filas:
                    celdas = fila_html.find_elements(By.TAG_NAME, "td")
                    if len(celdas) >= 3:
                        if celdas[1].text.strip() == anio_excel and celdas[2].text.strip() == semestre_excel:
                            boton_verificar = fila_html.find_element(By.XPATH, ".//span[text()='Verificar']/ancestor::button")
                            boton_verificar.click()
                            print(f"✅ Año {anio_excel} y semestre {semestre_excel} encontrados")
                            fila_encontrada = True
                            break
                
                if fila_encontrada:
                    break
                    
            except StaleElementReferenceException:
                print(f"⚠️ Intento {intento_anio + 1}: Elemento desactualizado en búsqueda Año/Semestre. Reintentando...")
                time.sleep(0.5)
            except Exception as e:
                print(f"⚠️ Error en intento {intento_anio + 1} para Año/Semestre: {e}")
                time.sleep(0.5)
        
        if not fila_encontrada:
            print(f"❌ No se encontró Año {anio_excel}, Semestre {semestre_excel} después de 3 intentos")
            # Marcar todas las cédulas del grupo como error
            for index in grupo.index:
                df.at[index, "ESTADO"] = "Error - Año/Semestre no encontrado"
            continue
        
        # 2. Buscar y hacer clic en PRO_CONSECUTIVO con reintentos
        encontrado_pro = False
        for intento_pro in range(3):
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]//tr[td]"))
                )
                
                # Refrescar la búsqueda de filas en cada intento
                filas_nuevas = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-hover')]//tr")
                
                for fila_html in filas_nuevas:
                    celdas = fila_html.find_elements(By.TAG_NAME, "td")
                    if len(celdas) >= 2:
                        if celdas[1].text.strip() == pro_consecutivo_excel:
                            boton_verificar = fila_html.find_element(By.XPATH, ".//span[text()='Verificar']/ancestor::button")
                            boton_verificar.click()
                            print(f"✅ PRO_CONSECUTIVO {pro_consecutivo_excel} encontrado")
                            encontrado_pro = True
                            break
                
                if encontrado_pro:
                    break
                    
            except StaleElementReferenceException:
                print(f"⚠️ Intento {intento_pro + 1}: Elemento desactualizado en búsqueda PRO_CONSECUTIVO. Reintentando...")
                time.sleep(0.7)
            except Exception as e:
                print(f"⚠️ Error en intento {intento_pro + 1} para PRO_CONSECUTIVO: {e}")
                time.sleep(0.5)
        
        if not encontrado_pro:
            print(f"❌ No se encontró PRO_CONSECUTIVO {pro_consecutivo_excel} después de 3 intentos")
            # Marcar todas las cédulas del grupo como error
            for index in grupo.index:
                df.at[index, "ESTADO"] = "Error - PRO_CONSECUTIVO no encontrado"
            continue
        
        # 3. Esperar que cargue la tabla de personas y procesar todas las cédulas del grupo
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//tbody[@id='form:checkboxDT_data']//tr"))
        )
        
        # Procesar cada cédula del grupo actual
        for index, fila in grupo.iterrows():
            documento_excel = str(int(fila["NUM_DOCUMENTO"])).strip()
            print(f"  🔍 Procesando cédula: {documento_excel}")
            
            try:
                # Buscar TODAS las filas que coincidan con la cédula
                filas_documentos = driver.find_elements(By.XPATH, "//tbody[@id='form:checkboxDT_data']//tr")
                
                filas_coincidentes = []
                for fila_html in filas_documentos:
                    celdas = fila_html.find_elements(By.TAG_NAME, "td")
                    if len(celdas) >= 4:
                        documento_actual = celdas[3].text.strip()
                        if documento_actual == documento_excel:
                            filas_coincidentes.append(fila_html)
                
                if not filas_coincidentes:
                    print(f"    ❌ Cédula {documento_excel} no encontrada en esta tabla")
                    df.at[index, "ESTADO"] = "Error - No encontrada"
                    cedulas_procesadas_grupo += 1
                    continue
                
                print(f"    🔍 Encontradas {len(filas_coincidentes)} filas con la cédula {documento_excel}")
                
                # Buscar cuál de las filas coincidentes tiene el botón "Sin Verificar"
                procesado = False
                for i, fila_html in enumerate(filas_coincidentes):
                    try:
                        # Verificar si esta fila tiene el botón "Sin Verificar"
                        boton_actualizar = fila_html.find_element(By.XPATH, ".//span[text()='Sin Verificar']/ancestor::button")
                        
                        print(f"    ✅ Cédula {documento_excel} encontrada con 'Sin Verificar' (fila {i+1}/{len(filas_coincidentes)})")
                        
                        # Clic en 'Sin Verificar'
                        boton_actualizar.click()
                        
                        time.sleep(1.3)  # Esperar diálogo
                        
                        # Clic en 'Si'
                        boton_si = driver.find_element(By.XPATH, "//span[text()='Si']/ancestor::button")
                        driver.execute_script("arguments[0].click();", boton_si)
                        
                        time.sleep(0.8)  # Esperar procesamiento
                        
                        df.at[index, "ESTADO"] = "OK"
                        cedulas_exitosas_grupo += 1
                        procesado = True
                        print(f"    ✅ Cédula {documento_excel} procesada exitosamente")
                        break
                        
                    except:
                        # Esta fila no tiene "Sin Verificar", continuar con la siguiente
                        print(f"    ⚠️ Fila {i+1} no tiene 'Sin Verificar', probando siguiente...")
                        continue
                
                if not procesado:
                    print(f"    ❌ Ninguna fila de la cédula {documento_excel} tiene 'Sin Verificar'")
                    df.at[index, "ESTADO"] = "Error - Sin botón 'Sin Verificar'"
                
                cedulas_procesadas_grupo += 1
                
            except Exception as e:
                print(f"    ❌ Error procesando cédula {documento_excel}: {e}")
                df.at[index, "ESTADO"] = "Error"
                cedulas_procesadas_grupo += 1
        
        print(f"📊 Grupo completado: {cedulas_exitosas_grupo}/{cedulas_procesadas_grupo} exitosas")
        
        # 4. Hacer "Volver" solo UNA VEZ por grupo (después de procesar todas las cédulas)
        # Volver desde tabla de personas
        try:
            boton_volver_personas = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-default') and contains(@onclick, 'mojarra.jsfcljs')]"))
            )
            driver.execute_script("arguments[0].click();", boton_volver_personas)
            print("🔙 Volver desde tabla de personas")
            time.sleep(0.5)
        except Exception as e:
            print(f"❌ Error en primer volver: {e}")
        
        # Volver desde tabla de carreras
        try:
            boton_volver_carreras = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//span[text()='Volver']/ancestor::button"))
            )
            driver.execute_script("arguments[0].click();", boton_volver_carreras)
            print("🔙 Volver desde tabla de carreras")
            time.sleep(0.5)
        except Exception as e:
            print(f"❌ Error en segundo volver: {e}")
        
    except Exception as e:
        print(f"❌ Error procesando grupo Año {anio_excel}, Semestre {semestre_excel}, PRO {pro_consecutivo_excel}: {e}")
        
        # Marcar como error solo las filas que no se hayan tocado
        for index in grupo.index:
            if df.at[index, "ESTADO"] == "Pendiente":
                df.at[index, "ESTADO"] = "Error - Falla general grupo"
        
        # Intentar recuperación: volver al menú o evitar quedarse trabado
        try:
            print("🔄 Intentando volver desde error general del grupo...")
    
            # Intentar volver desde personas
            try:
                boton_volver_personas = driver.find_element(By.XPATH, "//a[contains(@class, 'btn-default') and contains(@onclick, 'mojarra.jsfcljs')]")
                driver.execute_script("arguments[0].click();", boton_volver_personas)
                time.sleep(0.5)
                print("🔙 Regresado desde tabla de personas")
            except:
                pass
            
            # Intentar volver desde carreras
            try:
                boton_volver_carreras = driver.find_element(By.XPATH, "//span[text()='Volver']/ancestor::button")
                driver.execute_script("arguments[0].click();", boton_volver_carreras)
                time.sleep(0.5)
                print("🔙 Regresado desde tabla de carreras")
            except:
                pass
            
            # Si los botones no funcionaron, intentar verificar si se cayó al login
            current_url = driver.current_url
            if "login" in current_url:
                print("⚠️ Nos redirigió al login. No se puede recuperar este grupo.")
            else:
                try:
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]"))
                    )
                    print("✅ Ya estamos en la página correcta")
                except:
                    # Navegar al menú
                    try:
                        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "menu_33"))).click()
                        time.sleep(0.3)
                        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "menu_37"))).click()
                        time.sleep(0.3)
                        print("✅ Navegación de recuperación exitosa")
                    except:
                        print("❌ Falló la navegación de recuperación. Continuando con el siguiente grupo...")
        
        except Exception as nav_error:
            print(f"❌ Error en recuperación tras fallo general del grupo: {nav_error}")
            print("⚠️ Continuando con el siguiente grupo...")

# Guardar el resultado final
df[["NUM_DOCUMENTO", "PRO_CONSECUTIVO", "ESTADO"]].to_excel("resultado_procesamiento.xlsx", index=False)
print("📄 Archivo 'resultado_procesamiento.xlsx' guardado con éxito.")
print(f"📊 Resumen: {len(df[df['ESTADO'] == 'OK'])} exitosos, {len(df[df['ESTADO'] == 'Error'])} con errores")
# Cerrar el navegador
driver.quit()

📋 DataFrame ordenado por Año, Semestre y PRO_CONSECUTIVO - Total filas: 29
📊 Total de grupos a procesar: 16

🔄 Grupo 1/16: Año 2007, Semestre 1, PRO_CONSECUTIVO 1943
📋 Cédulas a procesar: 1 documentos
✅ Año 2007 y semestre 1 encontrados
⚠️ Intento 1: Elemento desactualizado en búsqueda PRO_CONSECUTIVO. Reintentando...
✅ PRO_CONSECUTIVO 1943 encontrado
  🔍 Procesando cédula: 94405355
    🔍 Encontradas 1 filas con la cédula 94405355
    ✅ Cédula 94405355 encontrada con 'Sin Verificar' (fila 1/1)
    ✅ Cédula 94405355 procesada exitosamente
📊 Grupo completado: 1/1 exitosas
🔙 Volver desde tabla de personas
🔙 Volver desde tabla de carreras

🔄 Grupo 2/16: Año 2007, Semestre 1, PRO_CONSECUTIVO 8936
📋 Cédulas a procesar: 2 documentos
✅ Año 2007 y semestre 1 encontrados
⚠️ Intento 1: Elemento desactualizado en búsqueda PRO_CONSECUTIVO. Reintentando...
✅ PRO_CONSECUTIVO 8936 encontrado
  🔍 Procesando cédula: 77029687
    🔍 Encontradas 1 filas con la cédula 77029687
    ✅ Cédula 77029687 encontra